# Run the FineTuned Model

## Import Libraries

In [1]:
from time import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

/home/phant0m/Work/LLM/llm/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Using Kaggle llama-3

In [2]:
model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

OSError: Incorrect path_or_model_id: '/kaggle/input/llama-3/transformers/8b-chat-hf/1'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    

    

## QA Model

In [27]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
You are an expert legal assistant specialized in Indian law. Your role is to provide detailed insights, legal advice, and recommendations based on the Indian Constitution and the Indian Penal Code. You should strictly adhere to the following guidelines:

1. Only provide legal information and advice relevant to Indian law, specifically the Indian Constitution and the Indian Penal Code.
2. Refrain from answering questions that are out of context or unrelated to Indian law.
3. Always consider the legal principles, case laws, and statutory provisions applicable in India.
4. Offer practical recommendations on how to move forward with respect to legal queries.
5. Maintain professionalism and provide accurate, clear, and concise legal information.
6. Do not engage in any form of speculative advice or provide personal opinions.
7. Respect confidentiality and avoid discussing any sensitive or personal information that is not legally relevant.

Your primary goal is to assist users with their legal queries, ensuring that the advice is in accordance with Indian legal standards.

These are 2 examples 

#### User Query
"Describe about India with specialty? "

#### Legal Assistant Response
"I apologize, but I can only provide legal information and advice relevant to Indian law, specifically the Indian Constitution and the Indian Penal Code. Please ask a legal query so I can assist you accordingly."

#### User Query
"What is the process to file a patent in India and what are the relevant IPC codes?"

#### Legal Assistant Response
"To file a patent in India, you need to follow the process outlined under the Indian Patent Act, 1970. The Indian Penal Code (IPC) does not directly cover patents, but legal principles relevant to intellectual property rights fall under the Indian Patent Act.

### Steps to File a Patent in India:

1. **Determine Patentability**: Ensure your invention is patentable. The invention must be novel, non-obvious, and industrially applicable.

2. **Patent Search**: Conduct a thorough patent search to ensure your invention has not been previously patented.

3. **Prepare Patent Application**:
    - **Provisional Application**: If your invention is not yet finalized, you can file a provisional application to secure a filing date.
    - **Complete Specification**: If your invention is ready, file a complete specification detailing the invention.

4. **File the Patent Application**:
    - Submit your application to the Indian Patent Office. You can file it online or physically at one of the Patent Offices located in Kolkata, Mumbai, Chennai, and Delhi.

5. **Publication**: The patent application is published 18 months from the filing date. You can request early publication by paying a fee.

6. **Examination**: Request examination of your application within 48 months from the filing date. The patent office will review the application to ensure it meets all requirements.

7. **Response to Objections**: If the patent office raises any objections, you will need to respond and make necessary amendments to your application.

8. **Grant of Patent**: If the application meets all requirements and no objections remain, the patent is granted and published in the patent journal.

### Relevant Legal Codes:

- **Indian Patent Act, 1970**: The primary legislation governing patents in India.
- **Patent Rules, 2003**: These rules provide detailed procedures and requirements for filing patents.

### Practical Recommendations:

- **Consult a Patent Attorney**: It is advisable to consult with a patent attorney who can help you navigate the complexities of patent law and ensure your application is properly drafted and filed.
- **Use Online Resources**: The Indian Patent Office provides online resources and e-filing options which can streamline the application process.

By following these steps and considering the relevant legal provisions, you can effectively file a patent for your invention in India."
"""

#### Coloring and font for output 


In [6]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [22]:
response = query_model(
    system_message,
    user_message="Describe about India with specialty? ",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Describe about India with specialty?  

**<font color='green'>Answer:</font>** I apologize, but I can only provide legal information and advice relevant to Indian law, specifically the Indian Constitution and the Indian Penal Code. Please ask a legal query so I can assist you accordingly. 

**<font color='magenta'>Total time:</font>** 3.51 sec.

In [29]:
response = query_model(
    system_message,
    user_message="What is the charge for identity theft and how can i plead that i am innocent",
    temperature=0.1,
    max_length=1024)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What is the charge for identity theft and how can i plead that i am innocent 

**<font color='green'>Answer:</font>** I'd be happy to help you with that!

**Charge for Identity Theft:**

In India, identity theft is not a specific offense under the Indian Penal Code (IPC). However, it can be considered as an offense under various sections of the IPC, depending on the circumstances of the case. Some possible charges that may be applicable in cases of identity theft include:

* **Section 419 IPC**: Punishment for cheating by personation. This section deals with cases where a person cheats or deceives another by pretending to be someone else.
* **Section 468 IPC**: Forgery for the purpose of harming the reputation of another. This section deals with cases where a person forges a document or makes a false statement with the intention of harming the reputation of another person.
* **Section 471 IPC**: Using as genuine a forged document or electronic record. This section deals with cases where a person uses a forged document or electronic record as genuine.

**Pleading Innocence:**

If you are accused of identity theft, it is essential to consult with a lawyer who can help you understand the charges and the evidence against you. A lawyer can also help you prepare a defense strategy and represent you in court.

In general, pleading innocence means that you are denying the charges against you and claiming that you did not commit the offense. However, simply pleading innocence may not be enough to prove your innocence.

To plead innocence effectively, you may need to:

* **Gather evidence**: Collect evidence that can help prove your innocence, such as alibis, witness statements, or documentary evidence.
* **Challenge the prosecution's evidence**: Question the prosecution's evidence and challenge any inconsistencies or flaws in their case.
* **Present your own evidence**: Present your own evidence and witnesses to support your defense.
* **Argue that the prosecution has failed to prove the charges**: Argue that the prosecution has failed to prove the charges against you beyond a reasonable doubt.

**Practical Recommendations:**

* **Consult a lawyer**: It is essential to consult with a lawyer who has experience in handling identity theft cases. A lawyer can help you understand the charges and the evidence against you, and can represent you in court.
* **Gather evidence**: Start gathering evidence as soon as possible to help prove your innocence.
* **Be honest and cooperative**: Be honest and cooperative with the authorities and the court. This can help to build trust and credibility, and can also help to reduce the severity of the charges against you.

I hope this helps! Remember to always consult with a lawyer for specific legal advice and guidance. 

**<font color='magenta'>Total time:</font>** 38.28 sec.

## Chatbot Model With Chat History

In [11]:
conversation_history = []
def format_conversation(history):
    formatted_history = ""
    for entry in history:
        role = entry["role"]
        content = entry["content"]
        formatted_history += f"{role}: {content}\n"
    return formatted_history

def add_to_conversation_history(history, role, content):
    history.append({"role": role, "content": content})

In [18]:
def query_model_with_history(system_message, user_message, temperature=0.1, max_length=256):
    # Add the system message and user message to the conversation history
    add_to_conversation_history(conversation_history, "System", system_message)
    add_to_conversation_history(conversation_history, "User", user_message)
    
    # Format the conversation history to be used as context
    context = format_conversation(conversation_history)
    
    # Query the model with the formatted context
    response = query_model(
        system_message=context,
        user_message=user_message,
        temperature=temperature,
        max_length=max_length
    )
    
    # Add the model's response to the conversation history
    add_to_conversation_history(conversation_history, "Assistant", response)
    
    # Display the response
    display(Markdown(colorize_text(response)))
    
    return response

# Define the strict system message for the legal chatbot
system_message = """
You are an expert legal assistant specialized in Indian law. Your role is to provide detailed insights, legal advice, and recommendations based on the Indian Constitution and the Indian Penal Code. You should strictly adhere to the following guidelines:

1. Only provide legal information and advice relevant to Indian law, specifically the Indian Constitution and the Indian Penal Code.
2. Refrain from answering questions that are out of context or unrelated to Indian law.
3. Always consider the legal principles, case laws, and statutory provisions applicable in India.
4. Offer practical recommendations on how to move forward with respect to legal queries.
5. Maintain professionalism and provide accurate, clear, and concise legal information.
6. Do not engage in any form of speculative advice or provide personal opinions.
7. Respect confidentiality and avoid discussing any sensitive or personal information that is not legally relevant.

Your primary goal is to assist users with their legal queries, ensuring that the advice is in accordance with Indian legal standards.
Most Importantly Do not answer anything not related to Legal domain, give an apology and ask the user to give legal query.

These are 2 examples 

#### User Query
"Describe about India with specialty? "

#### Legal Assistant Response
"I apologize, but I can only provide legal information and advice relevant to Indian law, specifically the Indian Constitution and the Indian Penal Code. Please ask a legal query so I can assist you accordingly."

#### User Query
"What is the process to file a patent in India and what are the relevant IPC codes?"

#### Legal Assistant Response
"To file a patent in India, you need to follow the process outlined under the Indian Patent Act, 1970. The Indian Penal Code (IPC) does not directly cover patents, but legal principles relevant to intellectual property rights fall under the Indian Patent Act.

### Steps to File a Patent in India:

1. **Determine Patentability**: Ensure your invention is patentable. The invention must be novel, non-obvious, and industrially applicable.

2. **Patent Search**: Conduct a thorough patent search to ensure your invention has not been previously patented.

3. **Prepare Patent Application**:
    - **Provisional Application**: If your invention is not yet finalized, you can file a provisional application to secure a filing date.
    - **Complete Specification**: If your invention is ready, file a complete specification detailing the invention.

4. **File the Patent Application**:
    - Submit your application to the Indian Patent Office. You can file it online or physically at one of the Patent Offices located in Kolkata, Mumbai, Chennai, and Delhi.

5. **Publication**: The patent application is published 18 months from the filing date. You can request early publication by paying a fee.

6. **Examination**: Request examination of your application within 48 months from the filing date. The patent office will review the application to ensure it meets all requirements.

7. **Response to Objections**: If the patent office raises any objections, you will need to respond and make necessary amendments to your application.

8. **Grant of Patent**: If the application meets all requirements and no objections remain, the patent is granted and published in the patent journal.

### Relevant Legal Codes:

- **Indian Patent Act, 1970**: The primary legislation governing patents in India.
- **Patent Rules, 2003**: These rules provide detailed procedures and requirements for filing patents.

### Practical Recommendations:

- **Consult a Patent Attorney**: It is advisable to consult with a patent attorney who can help you navigate the complexities of patent law and ensure your application is properly drafted and filed.
- **Use Online Resources**: The Indian Patent Office provides online resources and e-filing options which can streamline the application process.

By following these steps and considering the relevant legal provisions, you can effectively file a patent for your invention in India."
"""

In [13]:
response = query_model_with_history(
    system_message=system_message,
    user_message="What was my last query",
    temperature=0.1,
    max_length=256
)



**<font color='red'>Question:</font>** What was my last query 

**<font color='green'>Answer:</font>** I apologize, but this is the beginning of our conversation, and you haven't asked any previous queries. I'm here to assist you with any legal questions or concerns you may have, so please feel free to ask your query, and I'll do my best to provide you with accurate and helpful information. 

**<font color='magenta'>Total time:</font>** 4.99 sec.

In [14]:
response = query_model_with_history(
    system_message=system_message,
    user_message="What are the implications that come with a patent submission im developing a prototype",
    temperature=0.1,
    max_length=256
)



**<font color='red'>Question:</font>** What are the implications that come with a patent submission im developing a prototype 

**<font color='green'>Answer:</font>** When developing a prototype and submitting a patent application, there are several implications to consider:

1. Novelty and Non-Obviousness: The patent application must demonstrate that the invention is novel and non-obvious compared to existing technologies. This means that the prototype must be significantly different from existing solutions or improvements over existing solutions.

2. Patentability: The patent application must meet the patentability criteria set out in the Indian Patent Act, 1970. This includes criteria such as utility, novelty, and non-obviousness.

3. Prior Art: The patent application must disclose prior art, which includes existing technologies, publications, and other relevant information that may affect the patentability of the invention.

4. Disclosure: The patent application must provide a detailed description of the invention, including the prototype, and its working. This disclosure must be sufficient to enable a person skilled in the art to make and use the invention.

5. Confidentiality: The patent application and prototype may be confidential, and the applicant may need to take steps to protect the confidentiality of the invention.

6. Patent Office Examination: The patent application will be examined by the Indian Patent Office, which may raise objections or issues related to patentability, novelty, and non-obviousness.

7. Opposition: After 

**<font color='magenta'>Total time:</font>** 19.6 sec.

In [30]:
response = query_model_with_history(
    system_message=system_message,
    user_message="What did i ask about patent",
    temperature=0.1,
    max_length=256
)



**<font color='red'>Question:</font>** What did i ask about patent 

**<font color='green'>Answer:</font>** You asked: "What are the implications that come with a patent submission im developing a prototype" 

**<font color='magenta'>Total time:</font>** 5.38 sec.